# Overview

## Setup

In [6]:
import sagemaker

sess = sagemaker.session.Session()
bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

In [25]:
import sagemaker
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework
import sys
import os
import glob
import re
from IPython.display import HTML
sys.path.append("common")
from misc import wait_for_s3_object

In [26]:
estimator = RLEstimator(source_dir='src',
                        entry_point="train-coach.py",
                        dependencies=["common/sagemaker_rl"],
                        toolkit=RLToolkit.COACH,
                        toolkit_version='0.11.0',
                        framework=RLFramework.TENSORFLOW,
                        role=role,
                        train_instance_count=1,
                        train_instance_type='ml.m5.xlarge',
                        output_path='s3://{}/'.format(bucket),
                        base_job_name='rl-tic-tac-toe',
                        hyperparameters = {"RLCOACH_PRESET" : "preset"})
estimator.fit(wait=False)

INFO:sagemaker:Creating training-job with name: rl-tic-tac-toe-2019-06-04-18-49-39-760


In [55]:
job_name=estimator._current_job_name

output_tar_key = "{}/output/output.tar.gz".format(job_name)
intermediate_folder_key = "{}/output/intermediate".format(job_name)
output_url = "s3://{}/{}".format(bucket, output_tar_key)
intermediate_url = "s3://{}/{}".format(bucket, intermediate_folder_key)
tmp_dir = "/tmp/{}".format(job_name)
os.system("mkdir {}".format(tmp_dir))

256

In [56]:
key = intermediate_folder_key + '/gifs'
wait_for_s3_object(bucket, key, tmp_dir)    

glob_pattern = os.path.join("{}/*.gif".format(tmp_dir))
gifs = [file for file in glob.iglob(glob_pattern, recursive=True)]
extract_episode = lambda string: int(re.search('.*episode-(\d*)_.*', string, re.IGNORECASE).group(1))
gifs.sort(key=extract_episode)

# visualize a specific episode
gif_index = -1 # since we want last gif
gif_filepath = gifs[gif_index]
gif_filename = os.path.basename(gif_filepath)
print("Selected GIF: {}".format(gif_filename))
os.system("mkdir -p ./src/tmp_render/ && cp {} ./src/tmp_render/{}.gif".format(gif_filepath, gif_filename))
HTML('<img src="./src/tmp_render/{}.gif">'.format(gif_filename))

Waiting for s3://sagemaker-us-west-2-345362745630/rl-tic-tac-toe-2019-06-04-18-49-39-760/output/intermediate/gifs...
Selected GIF: 2019-06-04-18-52-57_episode-1640_score-1.0.gif
